In [2]:
import pandas as pd
import numpy as np
import holidays

In [5]:
# Merging subway clean (from Google Sheet - https://goo.gl/2WQxkV) to turnstile data
subway_clean_fp = 'subway_clean_raw.csv'
subway_clean = pd.read_csv(subway_clean_fp)
subway_clean = subway_clean.drop(columns=['Location','Comment'])

In [4]:
# Exporting clean subway coord data, keeping only single station name
subway_clean.sort_values(['ID','Station'],inplace=True)

master_names = subway_clean.groupby('ID').agg({'Station':lambda x: x.iloc[0],
                               'Line Name':lambda x: x.iloc[0],
                                'Division':lambda x: x.iloc[0],
                                'Latitude':lambda x: x.iloc[0],
                                'Longitude':lambda x: x.iloc[0]
                               })
# Add back in line 1 to Chamber St, Change name
master_names.loc[139,'Line Name'] = 'ACE123'
master_names.loc[146,'Station'] = 'DELANCEY/ESSEX ST'
master_names.loc[20,'Station'] = '14 ST/8 AV'
# # Export this if needed
# master_names.to_csv('subway_coords_clean.csv',encoding='utf-8')

In [6]:
# From NY State Open Data
turnstile_fp = 'Turnstile_Usage_Data__2015.csv'
turnstile = pd.read_csv(turnstile_fp)

In [7]:
t_sc = turnstile.merge(subway_clean,how='inner',left_on=['Station','Line Name','Division'],
                       right_on=['Station','Line Name','Division'])

In [8]:
# Somehow the above merge results in duplicate traffic data
dedupe = t_sc.drop_duplicates(subset=['Unit','SCP','Date','Time'])

In [9]:
# Generate date_time column
space = np.full(dedupe.shape[0],' ',dtype=str)
dedupe['date_time'] = pd.to_datetime(dedupe.Date + space + dedupe.Time,format="%m/%d/%Y %H:%M:%S")

/Users/dwoo/virtualenvs/dsi/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
# No longer necessary
dedupe = dedupe.drop(columns='Date')
# Bizzare column name cleanup
dedupe.rename(columns={'Exits                                                     ':'Exits'},inplace=True)

In [11]:
# Sort values first
dedupe.sort_values(['Unit','SCP','Station','Line Name',
                   'ID','Latitude','Longitude','date_time'],inplace=True)
timed = dedupe.groupby(['Unit','SCP','Station','Line Name','ID','Latitude','Longitude',
                pd.Grouper(key='date_time',freq='4h',closed='left',base=0)])\
                .agg({'Entries':lambda x: x.iloc[0],
                      'Exits':lambda x: x.iloc[0]}).reset_index()

In [12]:
timed['date_time_next'] = timed.date_time + pd.Timedelta('4h')

In [13]:
# Merging by offset time column to calculate delta
timed = timed.merge(timed,how='inner',left_on=['Unit','SCP','Station','Line Name','ID','Latitude','Longitude','date_time'],
            right_on=['Unit','SCP','Station','Line Name','ID','Latitude','Longitude','date_time_next'],
            suffixes=('','_prior'))

timed['entry_volume'] = timed.Entries - timed.Entries_prior
timed['exit_volume'] = timed.Exits - timed.Exits_prior

### **Quick note on issue with data**

There are negative entry/exit volumes, as well as exorbitantly high entry/exit volumes as well

The negative values seem to come from a couple possibilities:
1. Some turnstiles record cumulative entry/exits in seemingly reverse chronological order (see example below)
`dedupe[(dedupe.Unit == 'R011') & (dedupe.SCP == '00-00-04') & (dedupe.Date == '03/05/2015') & (dedupe.Station == '42 ST-PA BUS TE') & (dedupe['Line Name'] == 'ACENQRS1237')]`

2. Some turnstiles seem to reset their counters, where the cumulative record sees a steep drop all of a sudden
`dedupe[(dedupe.Unit == 'R548') & (dedupe.SCP == '00-00-01') & (dedupe.Date == '07/22/2015') & (dedupe.Station == 'CHRISTOPHER ST') & (dedupe['Line Name'] == '1')]`

To deal with this, I've done the following:
- Convert all negative entry/exit volumes into positive values (assume the reverse records are mistakes)
- Drop all values that are above the 0.9999 quantile (which is around 3000/4hr/device, or around 13/min/device), by replacing them with np.NaN.

In [14]:
# Cleaning
timed['entry_volume'] = timed.entry_volume.abs()
timed['exit_volume'] = timed.exit_volume.abs()

extreme_entry = timed.entry_volume.quantile(0.9999)
extreme_exit = timed.exit_volume.quantile(0.9999)

timed['entry_volume'] = np.where(timed.entry_volume > extreme_entry,np.NaN,timed.entry_volume)
timed['exit_volume'] = np.where(timed.exit_volume > extreme_exit,np.NaN,timed.exit_volume)

In [15]:
# Using same logic for cleaning up subway coordinates file to keep 1 row per station/time
consolidated = timed.groupby(['ID','date_time','date_time_prior']).agg(
                            {'entry_volume':sum,
                            'exit_volume':sum
                            }).reset_index()

In [16]:
# Add station attributes from master dataset of names
consolidated['station'] = consolidated.ID.map(master_names['Station'])
consolidated['lines'] = consolidated.ID.map(master_names['Line Name'])
consolidated['division'] = consolidated.ID.map(master_names['Division'])
consolidated['latitude'] = consolidated.ID.map(master_names['Latitude'])
consolidated['longitude'] = consolidated.ID.map(master_names['Longitude'])

In [17]:
# Adding some time attributes
consolidated['day_of_week'] = consolidated.date_time.dt.dayofweek
consolidated['day'] = consolidated.date_time.dt.weekday_name
consolidated['hour'] = consolidated.date_time.dt.hour
consolidated['date'] = consolidated.date_time.dt.date
consolidated['interval'] = consolidated.date_time_prior.dt.strftime(date_format="%I%p")+\
                            np.full(consolidated.shape[0],'-')+\
                            consolidated.date_time.dt.strftime(date_format="%I%p")
us_holidays = holidays.UnitedStates()
consolidated['is_holiday'] = consolidated.date_time.dt.date.apply(lambda x: x in us_holidays)

In [18]:
reordered_columns = ['date','date_time','interval','day_of_week','day','hour','is_holiday',
                     'ID','station','lines','entry_volume','exit_volume']
master_volume = consolidated[reordered_columns].rename(columns={'ID':'station_id'})

In [19]:
master_volume.to_csv('2015_manhattan_turnstile_usage.csv',index=False)